# Block Model with a 1D background

### => Question 1: Should we make it VTI? Can all the codes handle VTI resistivity?

### => Question 2: Currently x-directed dipole, 200 m long. Could make that an arbitary rotated dipole - can all codes handle arbitrary rotated dipoles? 

### Characteristics
- **Layered background** can be compared to 1D codes.
- **Simple blockmodel** as benchmark
- The blocks are an **adjusted version of the Dublin Test Model 1** (DTM1), Miensopust et al., 2013. Changes to the model include:
  - Added a layered background (instead of homogeneous background).
  - Converted to a shallow marine scenario, where the airwave is still dominantly present.
  - Layered background contains a weak, shallow resistor (e.g., tiny gas accumulations)
    and a deep strong resistor (e.g., basalt).
  - The blocks are made smaller in size (sort of MT to CSEM adaption):
      - all horizontal distances were divided by a factor of 5,
      - vertical distances by a factor of 10.
  
  
#### References
- Miensopust, M. P., P. Queralt, A. G. Jones, and the 3D MT modellers, 2013, Magnetotelluric 3-D inversion – a review of two successful workshops on forward and inversion code testing and comparison: Geophysical Journal International, 193, 1216–1238; DOI: [10.1093/gji/ggt066](https://doi.org/10.1093/gji/ggt066).

In [ ]:
import emg3d         # 3D modeller emg3d
import empymod       # 1D modeller for comparison
import discretize    # for the meshing
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib notebook
plt.style.use('ggplot')

## «CSEM Test Model 1»

In [ ]:
# Define all the required edges in x, y, and z-directions
hx, x0 = np.array([5500, 4000, 500, 500, 4000, 5500]), -10000
hy, y0 = np.array([6000, 1000, 3000, 3000, 1000, 6000]), -10000
hz, z0 = np.array([500, 100, 2500, 500, 1500, 200, 50, 250, 400, 500]), -6000

# Initiate the mesh
mesh = discretize.TensorMesh([hx, hy, hz], x0=np.array([x0, y0, z0]))

# Extract the depths of the layers (extracted from mesh, to ensure they are equal)
depth = np.r_[mesh.vectorNz[1], mesh.vectorNz[-5:-1]][::-1]

print("x-edges:", mesh.vectorNx)
print("y-edges:", mesh.vectorNy)
print("z-edges:", mesh.vectorNz)  # Includes background layers and blocks!
print("depths :", depth)         # Only layers

In [ ]:
# 1.A Initiate the resistivity model
res = np.zeros(mesh.nC)

# 1.B Define layered background model
res1d = [1e8, 0.3, 1, 5, 1, 1000]
res[mesh.gridCC[:, 2] > depth[0]] = res1d[0]     # Air
res[mesh.gridCC[:, 2] <= depth[0]] = res1d[1]    # Water
res[mesh.gridCC[:, 2] <= depth[1]] = res1d[2]    # Background
res[mesh.gridCC[:, 2] <= depth[-1]] = res1d[-1]  # Basement

# Add a shallow layer with slightly higher resistivities (5 Ohm.m)
res[(mesh.gridCC[:, 2] <= depth[1]) & (mesh.gridCC[:, 2] > depth[2])] = res1d[3]

# 1.C Add the three blocks
resistivities = [100, 10, 50]  # Resistivities Block 1, 2, and 3
block1 = [ -500,  500, -4000, 4000,  -2400, -900]
block2 = [    0, 5000, -3000,    0,  -2900, -2400]
block3 = [-5000,    0,     0, 3000, -5400, -2400]
for i, block in enumerate([block1, block2, block3]):
    block_i = (
        (mesh.gridCC[:, 0] >= block[0]) & (mesh.gridCC[:, 0] <= block[1]) &
        (mesh.gridCC[:, 1] >= block[2]) & (mesh.gridCC[:, 1] <= block[3]) &
        (mesh.gridCC[:, 2] >= block[4]) & (mesh.gridCC[:, 2] <= block[5])
    )

    res[block_i] = resistivities[i]

### Define survey

In [ ]:
# Source: x-directed electric dipole of 200 m length, 50 meter above seafloor
src = [-100, 100, 0, 0, -350, -350]
strength = 1  # 1 A source
freq = 1      # 1 Hz frequency

# 3 Receiver lines
rec_x = np.linspace(-10, 10, 101)*1e3
rec_y = np.array([-3, 0, 3])*1e3
rec_z = -400

In [ ]:
# Plot the resistivity model
mesh.plot_3d_slicer(np.log10(res), clim=np.log10([0.2, 200]))
fig = plt.gcf()
axs = fig.get_children()[1]
axs.plot(rec_x, np.ones(rec_x.size)*rec_y[0], 'r.')
axs.plot(rec_x, np.ones(rec_x.size)*rec_y[1], 'r.')
axs.plot(rec_x, np.ones(rec_x.size)*rec_y[2], 'r.')
axs.plot(src[0], src[1], 'w*')
plt.show()

## 1D modelling of layered background

In [ ]:
epm_1d = np.zeros((rec_x.size, 2), dtype=complex)
for i, y in enumerate(rec_y[:2]):
    epm_1d[:, i] = empymod.bipole(
        [*src[:4], -src[4], -src[5]],                   # empymod has
        [rec_x, np.ones(rec_x.size)*y, -rec_z, 10, 0],  # positive z
        -depth,                                         # downwards
        res1d, freq, srcpts=5, verb=2, htarg={'pts_per_dec': -1})

In [ ]:
plt.figure(figsize=(9, 5))

# Real
ax1 = plt.subplot(121)
plt.title(r'$\Re(E_x)$')
for i, y in enumerate(rec_y[:2]):
    plt.plot(rec_x, epm_1d[:, i].real, f'C{i}-', label=f'y = {y/1e3} km')
    plt.plot(rec_x, -epm_1d[:, i].real, f'C{i}--')
    
plt.yscale('log')
plt.legend()
plt.xlabel('Offset (km)')
plt.ylabel('Amplitude (V/m)')

# Imaginary
plt.subplot(122, sharey=ax1)
plt.title(r'$\Im(E_x)$')
for i, y in enumerate(rec_y[:2]):
    plt.plot(rec_x, epm_1d[:, i].imag, f'C{i}-', label=f'y = {y/1e3} km')
    plt.plot(rec_x, -epm_1d[:, i].imag, f'C{i}--')

plt.ylim([1e-16, 1e-11])
plt.legend()
plt.xlabel('Offset (km)')
plt.ylabel('Amplitude (V/m)')

plt.show()

## 3D modelling with emg3d, layered background

In [ ]:
empymod.Report([discretize])